In [5]:
ls ../models_neural/quote_attribution/

__init__.py                          train.py
__pycache__/                         utils_data_processing_helpers.py
classification_models.py             utils_dataset.py
config-sentence-classification.json  utils_lightning.py
data/                                utils_model_loader.py
eval.py                              utils_params.py
language_models.py                   utils_parser.py
scripts/


In [6]:
import pandas as pd 

In [10]:
real_training_data = pd.read_csv('../models_neural/quote_attribution/data/our-annotated-data__stage-2.tsv', sep='\t', header=None)

In [26]:
real_training_data[3].drop_duplicates().str.split('/').str.get(1).value_counts()

train    251
test      84
Name: 3, dtype: int64

In [36]:
real_training_data[1].apply(lambda x: 'None' if x == 'None' else 'Source').value_counts()

None      8451
Source    7342
Name: 1, dtype: int64

In [18]:
# make fake data

In [54]:
from random import random, randint, choice

In [28]:
num_train_docs = 251
num_test_docs = 84

In [69]:
sources = ['A', 'B', 'C', 'D', 'E']
source_sentences = [
    'He doesn’t teach math.',
    'You don’t have children.',
    'Does he play tennis?',
    'The train leaves every morning at 18 AM.',
    'Water freezes at 0°C',
    'I love my new pets.',
    'She doesn’t use a computer.',
    'It snows a lot in winter in Russia.',
    'We live in Texas.',
    'You go to holiday every summer.',
    'Do you like spaghetti?',
    'My daughter does the laundry.',
    'The train does not leave at 12 AM.',
    'I have no money at the moment.',
    'Do they talk a lot?',
    'Tomorrow early morning first I go to morning walk.',
    'We drink coffee every morning.',
    'I and my sister don’t see each other anymore.',
    'They don’t go to school tomorrow.',
    'Every child likes an ice cream.',
]

background_sentences = [
    'They have been known to say that.',
    'The path forward is unclear.',
    'This occurred as the opposite was being discussed.',
    'No known follow up occurred.',
    'For all of history, this has been said.',
    'The reasons were not apparent.',
    'Updates will follow.',
    'Research has shown the opposite.',
    'Daily modes are hard to establish.',
    'All sources declined comment',
]




output_docs = []
for i in range(num_train_docs + num_test_docs):
    doc_idx = ('/train/%s' if i < num_train_docs else '/test/%s') % i 
    num_sents = randint(20, 80)
    output_doc = []
    for s in range(num_sents):
        has_source = random() > .5
        if has_source:
            source = choice(sources)
            sent = choice(source_sentences)
            sent = '"%s" said %s' % (sent, source)
        else:
            source = 'None'
            sent = choice(background_sentences)

        output_docs.append([sent, source, s, doc_idx])

In [71]:
ls ../models_neural/quote_attribution/data

our-annotated-data__stage-2.tsv


In [77]:
pd.DataFrame(output_docs).to_csv(
    '../models_neural/quote_attribution/data/quote-attribution-classification__sanity-check-data.tsv', 
    sep='\t', header=None, index=False
)